In [8]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [3]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/My Drive/archive.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [10]:
import tensorflow as tf
import pandas as pd
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPool2D, LeakyReLU, BatchNormalization, Dropout, Dense, InputLayer, Flatten
from keras.losses import BinaryCrossentropy
from keras.optimizers import Adam

In [5]:
gen = ImageDataGenerator(rescale=1./255,validation_split = 0.2,zoom_range=(0.99,0.99),dtype=None)

In [6]:
train = gen.flow_from_directory("/tmp/Brain Tumor Data Set/Brain Tumor Data Set",
                               target_size = (150,150),
                               batch_size = 256,
                               class_mode = "binary",
                               color_mode = "rgb",
                               shuffle = True,
                               seed = 123,
                               subset = "training")

Found 3681 images belonging to 2 classes.


In [7]:
val = gen.flow_from_directory("/tmp/Brain Tumor Data Set/Brain Tumor Data Set",
                               target_size = (150,150),
                               batch_size = 8,
                               class_mode = "binary",
                               color_mode = "rgb",
                               shuffle = True,
                               seed = 123,
                               subset = "validation")

Found 919 images belonging to 2 classes.


In [11]:
model = keras.Sequential()
model.add(InputLayer(input_shape=(150,150,3)))
model.add(Conv2D(filters=32,kernel_size=3, activation="relu", padding="same"))
model.add(MaxPool2D())
model.add(Conv2D(filters=64,kernel_size=3, activation="relu", padding="same"))
model.add(MaxPool2D())


model.add(Flatten())


model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate=0.3))
model.add(Dense(64, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(rate=0.3))
model.add(Dense(1, activation="sigmoid"))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 87616)             0         
                                                                 
 dense (Dense)               (None, 128)               1

In [21]:
model.compile(optimizer=Adam(0.001),loss = BinaryCrossentropy(),metrics=['accuracy'])

In [22]:
from keras import utils, callbacks
earlystopping = callbacks.EarlyStopping(monitor="val_loss", mode="min",
                                        patience=5, restore_best_weights = True)

In [23]:
history = model.fit(train,verbose=1,callbacks = [earlystopping],epochs=15,validation_data=(val))

Epoch 1/15
15/15 [==============================] - 32s 2s/step - loss: 0.6100 - accuracy: 0.7139 - val_loss: 0.6409 - val_accuracy: 0.7693
Epoch 2/15
15/15 [==============================] - 32s 2s/step - loss: 0.3551 - accuracy: 0.8462 - val_loss: 0.5954 - val_accuracy: 0.6714
Epoch 3/15
15/15 [==============================] - 30s 2s/step - loss: 0.2054 - accuracy: 0.9234 - val_loss: 0.3672 - val_accuracy: 0.8411
Epoch 4/15
15/15 [==============================] - 35s 2s/step - loss: 0.1183 - accuracy: 0.9660 - val_loss: 0.2186 - val_accuracy: 0.9597
Epoch 5/15
15/15 [==============================] - 36s 2s/step - loss: 0.0731 - accuracy: 0.9818 - val_loss: 0.2108 - val_accuracy: 0.9608
Epoch 6/15
15/15 [==============================] - 30s 2s/step - loss: 0.0463 - accuracy: 0.9886 - val_loss: 0.1645 - val_accuracy: 0.9848
Epoch 7/15
15/15 [==============================] - 30s 2s/step - loss: 0.0312 - accuracy: 0.9943 - val_loss: 0.1665 - val_accuracy: 0.9946
Epoch 8/15
15/15 [==

In [34]:
model.save('brain_tumor.keras')

In [35]:
trained_model = keras.models.load_model('/content/brain_tumor.keras')

In [2]:
import matplotlib.pyplot as plt

In [3]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

NameError: ignored